In [1]:
import pandas as pd
import pdb
import numpy as np
from sklearn.cross_validation import train_test_split
from nltk.corpus import stopwords
from nltk import sent_tokenize
from spacy.en import English
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
parser = English()
lmtzr = WordNetLemmatizer()
porter = PorterStemmer() 

/Users/himavarsha/Envs/sklearn/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
themes_data_frame = pd.read_excel("Hotels_batch1_2_sent_level_retagged_train_data.xlsx", "Classes", index_col=None, na_values=['NA'])

In [3]:
classes = themes_data_frame['classes'].tolist()

In [4]:
data = pd.read_excel("Hotels_batch1_2_sent_level_retagged_train_data.xlsx", "TrainingData", index_col=None, na_values=['NA'])

In [5]:
mydoclist1 = []
id_series = []
label_series = []
cleaned_id_series = []
cleaned_label_series = []
ratings = []
themes_col = 'gl_themes'
single_theme = 0
split_char = "|"
features_col = "sentence"
id = "sent_id"
test_size = 0.2

In [6]:
for index, row in data.iterrows():
    labels = []
    theme = themes_col
#         print index
    if row[theme]==row[theme]:
        if (single_theme==1) and row[features_list[0]]!='NA':
            try:
                ind = classes.index(str(row[theme]))
                label_series.append(ind)
                mydoclist1.append(row[features_list[0]])
                if 'ratings' in row:
                    ratings.append(row['ratings'])
                id_series.append(row[id])
            except ValueError as ve:
                print("Row with index = "+str(index)+" : "+str(row[theme])+" : "+str(ve.message))
        elif row[features_col]!='NA':
            row_labels = str(row[theme]).split(split_char)
            for label in row_labels:
                try:
                    ind = classes.index(label)
                    labels.append(ind)
                except ValueError as ve:
                    print("Row with index = "+str(index)+" : "+str(label)+" : "+str(ve.message))
    #                 labels.append(label)

    if single_theme==1:
        if (len(labels)==1) and row[features_col]!='NA':
            mydoclist1.append(row[features_col])
            label_series.append(labels[0])
            if 'ratings' in row:
                ratings.append(row['ratings'])
            id_series.append(row[id])
    else:
        if ((labels)!=[]) and row[features_col]!='NA':
            mydoclist1.append(row[features_col])
            label_series.append(labels)
            if 'ratings' in row:
                ratings.append(row['ratings'])
            id_series.append(row[id])
print('the total number of documents are', len(mydoclist1))
print('the total number of labels are', len(label_series))


/Users/himavarsha/Envs/sklearn/lib/python2.7/site-packages/ipykernel/__main__.py:23: DeprecationWarning: BaseException.message has been deprecated as of Python 2.6


Row with index = 88 : others : 'others' is not in list
Row with index = 122 : Heat during the day : 'Heat during the day' is not in list
Row with index = 210 : Private pool : 'Private pool' is not in list
Row with index = 247 : Network signals : 'Network signals' is not in list
Row with index = 278 : Laundry services : 'Laundry services' is not in list
Row with index = 282 : Boutique services : 'Boutique services' is not in list
Row with index = 289 : Private pool : 'Private pool' is not in list
Row with index = 320 : Private pool : 'Private pool' is not in list
Row with index = 331 : Private decks : 'Private decks' is not in list
Row with index = 344 : Private pool : 'Private pool' is not in list
Row with index = 345 : Private pool : 'Private pool' is not in list
Row with index = 362 : Discovery center : 'Discovery center' is not in list
Row with index = 405 : Discovery center : 'Discovery center' is not in list
Row with index = 416 : Discovery center : 'Discovery center' is not in li

In [7]:
# stoplist = stopwords.words("english") + ['blackberry','it\'s','i','a','an','is','my','to','it','the']
stoplist = []

In [8]:
import nltk
dp_pairs = ["dobj","advcl","ccomp","nmod","compound", "acl", "acomp", "csubj", "amod","nsubj"]
freq_dict = dict()
for example in mydoclist1:
    sentences = nltk.sent_tokenize(example)
    for sentence in sentences:
        try:
            parsedEx = parser(unicode(sentence))
        except:
            pdb.set_trace()
        # shown as: original token, dependency tag, head word, left dependents, right dependents
        for token in parsedEx:
            if token.dep_ in dp_pairs and not(token.orth_.lower() in stoplist) and not(token.head.orth_.lower() in stoplist):
                first = porter.stem(lmtzr.lemmatize(token.orth_.lower()))
                second = porter.stem(lmtzr.lemmatize(token.head.orth_.lower()))
                keyword = "_".join(sorted([first, second]))
                if keyword in freq_dict.keys():
                    freq_dict[keyword] = freq_dict[keyword] + 1
                else:
                    freq_dict[keyword] = 1
    

In [9]:
count = 0
texts = []
for example in mydoclist1:
    sentences = nltk.sent_tokenize(example)
    test = []
    for sentence in sentences:
        try:
            parsedEx = parser(unicode(sentence))
        except:
            pdb.set_trace()
        # shown as: original token, dependency tag, head word, left dependents, right dependents
        for token in parsedEx:
            if token.dep_ in dp_pairs and not(token.orth_.lower() in stoplist) and not(token.head.orth_.lower() in stoplist):
                first = porter.stem(lmtzr.lemmatize(token.orth_.lower()))
                second = porter.stem(lmtzr.lemmatize(token.head.orth_.lower()))
                keyword = "_".join(sorted([first, second]))
                if freq_dict[keyword] > 0:
                    test.append(keyword)
    count = count + 1
    texts.append(test)


In [10]:
cleaned_label_series = []
cleaned_id_series = []
cleaned_texts = []
for i in xrange(len(texts)):
    if texts[i] != []:
        cleaned_texts.append((" ").join(texts[i]))
        cleaned_label_series.append(label_series[i])
        cleaned_id_series.append(id_series[i])
    

In [11]:
## Split of the data into training and testing data
X_train, X_test, y_train,y_test, train_files, test_files  = train_test_split(
                                                                        np.array(cleaned_texts),
                                                                        np.array(cleaned_label_series),
                                                                        np.array(cleaned_id_series),
                                                                        test_size = test_size,
                                                                        random_state = 42)


In [12]:
texts[0]

[u'are_roger', u'awesom_host', u'have_they', u'staff_wonder', u'have_staff']

In [13]:
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()

In [14]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = mlb.fit_transform(y_train)


In [15]:
vectorizer = TfidfVectorizer(lowercase=True,
                                    use_idf=False,
                                    norm=u'l2' ,
                                    min_df=.00001,
                                    smooth_idf=False,
                                    analyzer='word',
                                    ngram_range=(1,1))
X_train1 = vectorizer.fit_transform(X_train)
X_test1 = vectorizer.transform(X_test)
clf = OneVsRestClassifier(LogisticRegression(C=(10.0), solver= 'lbfgs',penalty='l2')).fit(X_train1, y_train)

In [16]:
preds = clf.predict_proba(X_test1)

In [34]:
from sklearn.metrics import classification_report,average_precision_score
from sklearn.metrics.classification import precision_recall_fscore_support,confusion_matrix
from sklearn.preprocessing.label import label_binarize
from sklearn import metrics
import matplotlib.pyplot as plt
from math import ceil
from collections import OrderedDict
import json, simplejson
import os
from sklearn.preprocessing import MultiLabelBinarizer
import pdb
from decimal import Decimal

def dump_pred_prob(df_test,X_test_prob_array,columns_to_display,list_classes,OUTPUT_DIR):       
    pred_prob = []
    for i in xrange(len(list_classes)):
            title = 'pred_prob_'+list_classes[i]
            df_test[title] = X_test_prob_array[:,i]
            pred_prob.append('pred_prob_'+list_classes[i])
#        X_test_prob = (X_test_prob_array.tolist())
    filepath_temp = OUTPUT_DIR + "/" + "pred_prob_outputs.csv"
    columns_to_display.extend(pred_prob)
    df_test[columns_to_display].to_csv(filepath_temp)
    return df_test,pred_prob
def generate_report(y_test, y_pred_proba, list_classes,id_test=None,  y_pred=None, x_test=None,
                     multilabel = False, OPT_CONFIDENCE = 0.95,OUTPUT_DIR = 'Report_generator_results/'):

    
    id1 = 'id_test'
    x = 'X_test'
    y = 'y_test'
    if id_test==None:
        id_test = range(1,len(y_test)+1)
        train_transf_y = 'y_train_transformed'
    pred_y_2 = 'y_pred_test_2'
    prob_of_chosen = 'prob_of_chosen'
    pred_y_3 = 'y_pred_test_3'
    pred_prob = []
    mlb = MultiLabelBinarizer(classes=range(len(list_classes)))  
    df_test = pd.DataFrame()
    df_test[y] = y_test
    df_test[id1] = id_test
    subset = [id1]
    if x_test!=None:
        df_test[x] = x_test
        subset.append(x)
    if y_pred != None:
        df_test[pred_y_2] = y_pred
        subset.append(pred_y_2)
    
    if(y_pred!=None):
        overall_precision,overall_recall,overall_fscore,overall_support = precision_recall_fscore_support(df_test[y],
                                                df_test[pred_y_2],labels=range(len(list_classes)),average='weighted')
        print("Precision " + str(overall_precision) + "\n" + "Recall " + str(overall_recall))
        print("The accuracy of this classifier is :  ", str(metrics.accuracy_score(df_test[y], df_test[pred_y_2])))
        print(classification_report(df_test[y], df_test[pred_y_2], target_names=list_classes))

    ## Here it shows the overall accuracy as required by multilabel case
    ## For more reference see here :: http://scikit-learn.org/stable/modules/model_evaluation.html
    #y_test[0].append(y_test[0][0])
   
    ## Computing the prediction probabilities of the classifier
    X_test_prob_array = y_pred_proba
    df_test,pred_prob = dump_pred_prob(df_test, X_test_prob_array, subset, list_classes, OUTPUT_DIR)
    
    if multilabel:
        y_df_test = df_test[y]
        if len(y_df_test.shape) == 1 and not isinstance(y_df_test[0],list):
            y_df_test = y_df_test.reshape(y_df_test.shape[0],1)
        y_test_bin = mlb.fit_transform(y_df_test)
        map_score = average_precision_score(y_test_bin,df_test[pred_prob])
        map_score_macro = average_precision_score(y_test_bin,df_test[pred_prob],'macro')
        map_score_micro = average_precision_score(y_test_bin,df_test[pred_prob],'micro')
        map_score_weighted = average_precision_score(y_test_bin,df_test[pred_prob],'weighted')
        map_score_samples = average_precision_score(y_test_bin,df_test[pred_prob],'samples')
        fp5 = open(OUTPUT_DIR + "/" + "prec_recall.txt", "wb")
        print >> fp5, "MAP is " + str(map_score)
        print >> fp5, "MAP macro average is " + str(map_score_macro)
        print >> fp5, "MAP micro average is " + str(map_score_micro)
        print >> fp5, "MAP weighted average is  " + str(map_score_weighted)
        print >> fp5, "MAP samples average is " + str(map_score_samples)
        fp5.close()

    else:
        y_test_bin = label_binarize(df_test[y], classes=range(len(list_classes)))
    
    n_classes = y_test_bin.shape[1]
    precision = dict()
    recall = dict()
    thresholds = dict()
    n_classes_optimum_thresholds = OrderedDict()
    
    
    for i in range( n_classes ):
        #try:
        precision[i], recall[i], thresholds[i] = metrics.precision_recall_curve(y_test_bin[:,i], X_test_prob_array[:,i])
        max_thresh = 0
        thresh_found = False
        thresh_list = [1]
        for j, th in enumerate( thresholds[i] ):
            if(precision[i][j] > OPT_CONFIDENCE and not thresh_found):
                thresh_list.append(th)
                # thresh_found = True
        if not thresh_found:
            n_classes_optimum_thresholds[list_classes[i]]= Decimal(min(thresh_list))
        #subplt_num = i +1
        fig = plt.figure(i)
        subplt = fig.add_subplot(1,1,1)
        subplt.plot(recall[i],precision[i])
        subplt.set_title(list_classes[i])
        subplt.set_xlabel('Recall')
        subplt.set_ylabel('Precision')
        PREC_RECALL_DIR = OUTPUT_DIR + '/PREC_RECALL_PLOTS'
        if not (os.path.isdir(PREC_RECALL_DIR) or os.path.isfile(PREC_RECALL_DIR)) :
        	os.mkdir(PREC_RECALL_DIR)
        fig.savefig(OUTPUT_DIR +'/PREC_RECALL_PLOTS' + "/"+ list_classes[i].replace('/','-')+"_prec_recall_curve.png")
        plt.clf()
    
    # fig.savefig(OUTPUT_DIR + "/"+ "prec_recall_curves.png")
    ## Printing the precision-recall information for these cutoffs
    fp5 = open(OUTPUT_DIR + "/" + "prec_recall.txt", "ab")
    
    df_test[prob_of_chosen] = df_test[pred_prob].apply(max,axis=1)
    labels_ = range(len(list_classes))
    target_names_ = list(list_classes)
    if not multilabel:
        labels_.extend([-1])
        target_names_.extend(['none of them'])
    cls_series = pd.Series(pred_prob)
    pred_df = df_test[pred_prob]
    #pdb.set_trace()
    for cutoff in range(10)+[9.5]:
        cutoff_new = cutoff/10.0
        str_cutoff = 'cutoff_'+ str(cutoff)
        if multilabel:
            temp_df = pred_df.apply(lambda x, cut=cutoff_new, classes=cls_series: tuple(classes[classes.isin(x[x > cut].index)].index),axis =1)
            df_test[str_cutoff] = temp_df
#                 if cutoff == 0.5:
#                     conf_matrix = confusion_matrix(df_test[y],df_test[str_cutoff],labels=labels_)
        else:
            df_test[str_cutoff] = pred_df.apply(lambda x, cut=cutoff_new, cls=cls_series: cls[cls == x.idxmax()].index[0] if x.max() > cut else -1,axis=1)
            if cutoff == 0.0:
                conf_matrix = confusion_matrix(df_test[y],df_test[str_cutoff],labels=labels_)
        print >> fp5, "Cutoff is :: " , cutoff_new
        if multilabel:
            print >> fp5, "The accuracy of this classifier is :  ", str(metrics.accuracy_score(y_test_bin, mlb.fit_transform(df_test[str_cutoff])))
            print >> fp5, classification_report(y_test_bin, mlb.fit_transform(df_test[str_cutoff]), labels= labels_,target_names=target_names_)
        else:
            print >> fp5, "The accuracy of this classifier is :  ", str(metrics.accuracy_score(df_test[y], df_test[str_cutoff]))
            print >> fp5, classification_report(df_test[y], df_test[str_cutoff], labels= labels_,target_names=target_names_)
    if not multilabel:
        df_conf_matrix = pd.DataFrame(conf_matrix)
        df_conf_matrix.columns = target_names_
        df_conf_matrix.index = target_names_
        df_conf_matrix.to_csv(OUTPUT_DIR+'/'+'confusion_matrix_default.csv')
#         condition_95 = df_test[id] != ""
#         for i in range(len(list_classes_in_classifier)):
#             conditiontemp = (df_test[pred_prob[i]] == df_test[prob_of_chosen]) & (df_test[pred_prob[i]] > n_classes_optimum_thresholds[list_classes_in_classifier[i]])
#             condition_95 = condition_95 | conditiontemp
    def cutoff_func_multiclass_binary_95(x):
        cls_name = x.idxmax().split("_")[2]
        if cls_name in n_classes_optimum_thresholds:
            if x.max() > n_classes_optimum_thresholds[cls_name]:
                return cls_series[cls_series == x.idxmax()].index[0] 
            else:
                return -1
        elif x.idxmax() in cls_series.values:       ### for binary case
            return cls_series[cls_series == x.idxmax()].index[0]
        else:
            return -10
        
    def cutoff_func_multilabel_95(x):
        cls = cls_series
        thresh = pd.Series(n_classes_optimum_thresholds)
        thresh.index = x.index
        return tuple(cls[cls.isin(x[x > thresh].index)].index)
               
    str_95 = 'cutoff_'+ str('95_by_theme')
    if multilabel:
        df_test[str_95] = pred_df.apply(cutoff_func_multilabel_95, axis=1)
    else:   
        df_test[str_95] = pred_df.apply(cutoff_func_multiclass_binary_95, axis=1)
#         df_test[str_95] = df_test[pred_y_2]
#         if multilabel:
# 
#             df_test[str_95][~condition_95] = tuple([])
#         else:
#             df_test[str_95][~condition_95] = -1
#         y_real_95 = df_test[condition][y]
#         predicted3 = df_test[condition][pred_y_2]
    print >> fp5, "Cutoff is :: " + str(OPT_CONFIDENCE) + "% precision by Theme" ,
#         try: 
    if multilabel:
        print >> fp5, "The accuracy of this classifier is :  ", str(metrics.accuracy_score(y_test_bin, mlb.fit_transform(df_test[str_95])))
        print >> fp5, classification_report(y_test_bin, mlb.fit_transform(df_test[str_95]), labels= labels_,target_names=target_names_)
    else:
        print >> fp5, "The accuracy of this classifier is :  ", str(metrics.accuracy_score(df_test[y], df_test[str_95]))
        print >> fp5, classification_report(df_test[y], df_test[str_95], labels= labels_,target_names=target_names_)
        conf_matrix_2 = confusion_matrix(df_test[y], df_test[str_95], labels=labels_)
        df_conf_matrix_2 = pd.DataFrame(conf_matrix_2)
        df_conf_matrix_2.columns = target_names_
        df_conf_matrix_2.index = target_names_
        df_conf_matrix_2.to_csv(OUTPUT_DIR+'/'+'confusion_matrix_at_95prec.csv')
#             print >> fp5
#         except:
#             pdb.set_trace()
        
    fp5.close()
    if multilabel:
        _,recall_95_prec,_,_ =precision_recall_fscore_support(y_test_bin, mlb.fit_transform(df_test[str_95]), labels= labels_)
    else:
        _,recall_95_prec,_,_ =precision_recall_fscore_support(df_test[y], df_test[str_95], labels= labels_)
    
    
    txtfile2 = OUTPUT_DIR + "/" + "testing_final_results.txt"
    df_test.to_csv(txtfile2, sep = '\t')
    
    with open(OUTPUT_DIR+"/"+"class_thresholds.txt", "w") as json_file:
        #pdb.set_trace()
        simplejson.dump(n_classes_optimum_thresholds, json_file)
       

In [36]:
generate_report(y_test, preds, classes,id_test=test_files,  y_pred=None, x_test=X_test,
                     multilabel = True, OPT_CONFIDENCE = 0.95,OUTPUT_DIR = '.')

/Users/himavarsha/Envs/sklearn/lib/python2.7/site-packages/ipykernel/__main__.py:32: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
/Users/himavarsha/Envs/sklearn/lib/python2.7/site-packages/ipykernel/__main__.py:44: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


In [37]:
parsedEx = parser(unicode("game drives are the best!"))

In [39]:
for token in parsedEx:
    print token, token.orth_, token.dep_, token.head.orth_

game game compound drives
drives drives nsubj are
are are ROOT are
the the det best
best best attr are
! ! punct are
